In [2]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))


################################################################################
## General Stuff
################################################################################
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote
from collection import collections
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from webUtils import getHTML
from time import sleep
import time
from multiprocessing import Pool


################################################################################
## Music Stuff
################################################################################

## General
from discogsBase import discogs
from multiArtist import multiartist
from myMusicDBMap import myMusicDBMap

## Discogs
from artistDC import artistDC
from artistsDC import artistsDC
from discogsUtils import discogsUtils
from albums import albums

## AllMusic
from artistAM import artistAM
from artistsAM import artistsAM
from discogsUtils import allmusicUtils
from artistsAM import artistsAM

## MusicBrainz
from artistMB import artistMB
from artistsMB import artistsMB
from discogsUtils import musicbrainzUtils

## AceBootlegs
from artistAB import artistAB
from artistsAB import artistsAB
from discogsUtils import acebootlegsUtils

## RateYourMusic
from artistRM import artistRM
from artistsRM import artistsRM
from discogsUtils import rateyourmusicUtils

## LastFM
from artistLM import artistLM
from artistsLM import artistsLM
from discogsUtils import lastfmUtils

## DatPiff
from artistDP import artistDP
from artistsDP import artistsDP
from discogsUtils import datpiffUtils

## RockCorner
from artistRC import artistRC
from artistsRC import artistsRC
from discogsUtils import rockcornerUtils



import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-04-03 18:51:59.322189


# Collections

In [ ]:
%load_ext autoreload
%autoreload
#col = collections(disc)
#col.parseCollections(force=False)
#col.createCollectionDBs()

# Parse Artists (All Music)

In [ ]:
for i in range(100):
    artsAM.parseArtistModValFiles(i, force=True)

In [ ]:
artMB = artistMB(discMB)
retval = artMB.getData("/Volumes/Biggy/Discog/artists-musicbrainz/0/148150553282083715092252970758494088400.p")
retval.url.get()
retval.ID.get()

# Parse Artists (Parallel)

In [7]:
%load_ext autoreload
%autoreload
discAM   = discogs('allmusic')
artsAM   = artistsAM(discAM)
dutilsAM = allmusicUtils()

discDC   = discogs('discogs')
artsDC   = artistsDC(discDC)
dutilsDC = discogsUtils()

discMB   = discogs('musicbrainz')
artsMB   = artistsMB(discMB)
dutilsMB = musicbrainzUtils()

discAB   = discogs('acebootlegs')
artsAB   = artistsAB(discAB)
dutilsAB = acebootlegsUtils()

discRM   = discogs('rateyourmusic')
artsRM   = artistsRM(discRM)
dutilsRM = rateyourmusicUtils()

discLM   = discogs('lastfm')
artsLM   = artistsLM(discLM)
dutilsLM = lastfmUtils()

discDP   = discogs('datpiff')
artsDP   = artistsDP(discDP)
dutilsDP = datpiffUtils()

discRC   = discogs('rockcorner')
artsRC   = artistsRC(discRC)
dutilsRC = rockcornerUtils()


def parseArtistsAM(modVal, force=False, doExtra=False):
    artsAM.parseArtistModValFiles(modVal, force=force)

def parseArtistsDC(modVal, force=False, doExtra=False):
    artsDC.parseArtistModValFiles(modVal, force=force)

def parseArtistsMB(modVal, force=False, doExtra=False):
    artsMB.parseArtistModValFiles(modVal, force=True)

def parseArtistsAB(modVal, force=False, doExtra=False):
    artsAB.parseArtistFiles(force=force)
    
def parseArtistsDP(modVal, force=False, doExtra=False):
    artsDP.parseArtistFiles(force=force)

def parseArtistsRM(modVal, force=False, doExtra=False):
    artsRM.parseArtistModValFiles(modVal, force=force)

def parseArtistsLM(modVal, force=False, doExtra=False):
    artsLM.parseArtistModValFiles(modVal, force=force)

def parseArtistsRC(modVal, force=False, doExtra=False):
    artsRC.parseArtistModValFiles(modVal, force=True)

    
def parseArtistsParallel(db, nProcs=3, force=False):
    pool = Pool(processes=nProcs)
    if db == "Discogs":
        result = pool.map_async(parseArtistsDC, range(100))
    elif db == "AllMusic":
        result = pool.map_async(parseArtistsAM, range(100))
    elif db == "MusicBrainz":
        result = pool.map_async(parseArtistsMB, range(100))
    elif db == "AceBootlegs":
        result = pool.map_async(parseArtistsAB, [None])
    elif db == "DatPiff":
        result = pool.map_async(parseArtistsDP, [None])
    elif db == "RateYourMusic":
        result = pool.map_async(parseArtistsRM, range(100))
    elif db == "LastFM":
        result = pool.map_async(parseArtistsLM, range(100))
    elif db == "RockCorner":
        result = pool.map_async(parseArtistsRC, range(100))
    else:
        raise ValueError("[{0}] is not recognized as a DB".format(db))

    while not result.ready():
        if force is True:
            time.sleep(10)
        else:
            time.sleep(1)
    print("")
    return result.get()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#artsLM.parseArtistModValFiles(0, force=True)

In [ ]:
#getFile("/Volumes/Biggy/Discog/artists-lastfm-db/metadata/0-MediaMetadata.p")

In [ ]:
#artsDC.parseArtistModValFiles(29, force=True)
#arts.parseArtistModValFiles(50, force=True)
#artsLM.parseArtistModValFiles(96)

In [ ]:
ifile="/Volumes/Biggy/Discog/artists-lastfm/0/53902704000.p"
bsdata = getHTML(ifile)
artLM = artistLM()
#artLM.getData(ifile)

In [8]:
start,cmt=clock("Parsing All Artists")
start2,cmt2=clock("Parallel Artist Parsing")
#parseArtistsParallel(db="AllMusic", nProcs=3)
#parseArtistsParallel(db="MusicBrainz", nProcs=3)
#parseArtistsParallel(db="Discogs", nProcs=3)
#parseArtistsParallel(db="AceBootlegs", nProcs=1)
#parseArtistsParallel(db="DatPiff", nProcs=1)
#parseArtistsParallel(db="RateYourMusic", nProcs=4)
#parseArtistsParallel(db="LastFM", nProcs=3)
parseArtistsParallel(db="RockCorner", nProcs=3)
elapsed(start2, cmt2)
elapsed(start, cmt)

Current Time is Fri Apr 03, 2020 20:03:30 for Parsing All Artists
Current Time is Fri Apr 03, 2020 20:03:30 for Parallel Artist Parsing
Parsing Artist Files For ModVal 0
Parsing Artist Files For ModVal 9
Parsing Artist Files For ModVal 18
Forcing Reloads of ModVal=0
Forcing Reloads of ModVal=9
  Processing 19 files.
Forcing Reloads of ModVal=18
  Processing 14 files.
	Processed 0/19 files.
  Processing 9 files.
	Processed 0/14 files.
	Processed 0/9 files.
Saving 9 new artist IDs to /Volumes/Biggy/Discog/artists-rockcorner-db/18-DB.p
  --> This file is 31.1kB.
Saving 9 new artist IDs name data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/18-Metadata.p
  --> This file is 461B.
Saving 9 new artist IDs media data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/18-MediaMetadata.p
  --> This file is 24.2kB.
Parsing Artist Files For ModVal 19
Forcing Reloads of ModVal=19
  Processing 18 files.
	Processed 0/18 files.
Saving 14 new artist IDs to /Volumes/Biggy/Discog/artists-

Saving 10 new artist IDs media data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/14-MediaMetadata.p
  --> This file is 29.3kB.
Parsing Artist Files For ModVal 24
Forcing Reloads of ModVal=24
  Processing 14 files.
	Processed 0/14 files.
  --> This file is 35.4kB.
Parsing Artist Files For ModVal 15
Forcing Reloads of ModVal=15
  Processing 11 files.
	Processed 0/11 files.
Saving 14 new artist IDs to /Volumes/Biggy/Discog/artists-rockcorner-db/5-DB.p
  --> This file is 58.7kB.
Saving 14 new artist IDs name data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/5-Metadata.p
  --> This file is 654B.
Saving 14 new artist IDs media data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/5-MediaMetadata.p
  --> This file is 45.8kB.
Parsing Artist Files For ModVal 6
Forcing Reloads of ModVal=6
  Processing 11 files.
	Processed 0/11 files.
Saving 11 new artist IDs to /Volumes/Biggy/Discog/artists-rockcorner-db/15-DB.p
  --> This file is 43.0kB.
Saving 11 new artist IDs nam

  --> This file is 26.1kB.
Parsing Artist Files For ModVal 39
Forcing Reloads of ModVal=39
  Processing 11 files.
	Processed 0/11 files.
  --> This file is 26.4kB.
Parsing Artist Files For ModVal 47
Forcing Reloads of ModVal=47
  Processing 14 files.
	Processed 0/14 files.
Saving 11 new artist IDs to /Volumes/Biggy/Discog/artists-rockcorner-db/39-DB.p
  --> This file is 43.3kB.
Saving 11 new artist IDs name data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/39-Metadata.p
  --> This file is 536B.
Saving 11 new artist IDs media data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/39-MediaMetadata.p
Saving 17 new artist IDs to /Volumes/Biggy/Discog/artists-rockcorner-db/29-DB.p
  --> This file is 34.1kB.
Parsing Artist Files For ModVal 40
Forcing Reloads of ModVal=40
Saving 14 new artist IDs to /Volumes/Biggy/Discog/artists-rockcorner-db/47-DB.p
  Processing 11 files.
	Processed 0/11 files.
  --> This file is 56.0kB.
  --> This file is 68.7kB.
Saving 14 new artist IDs na

  --> This file is 393B.
Saving 8 new artist IDs media data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/44-MediaMetadata.p
  --> This file is 26.3kB.
Parsing Artist Files For ModVal 54
Forcing Reloads of ModVal=54
  Processing 13 files.
	Processed 0/13 files.
Saving 19 new artist IDs to /Volumes/Biggy/Discog/artists-rockcorner-db/34-DB.p
  --> This file is 80.5kB.
Saving 19 new artist IDs name data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/34-Metadata.p
  --> This file is 842B.
Saving 19 new artist IDs media data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/34-MediaMetadata.p
Saving 10 new artist IDs to /Volumes/Biggy/Discog/artists-rockcorner-db/53-DB.p
  --> This file is 63.9kB.
Parsing Artist Files For ModVal 35
Forcing Reloads of ModVal=35
  Processing 19 files.
	Processed 0/19 files.
  --> This file is 36.9kB.
Saving 10 new artist IDs name data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/53-Metadata.p
  --> This file is 459B.
Saving

Parsing Artist Files For ModVal 68
Forcing Reloads of ModVal=68
  Processing 13 files.
	Processed 0/13 files.
  --> This file is 40.9kB.
Saving 11 new artist IDs name data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/83-Metadata.p
  --> This file is 506B.
Saving 11 new artist IDs media data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/83-MediaMetadata.p
  --> This file is 31.9kB.
Parsing Artist Files For ModVal 84
Forcing Reloads of ModVal=84
  Processing 16 files.
	Processed 0/16 files.
Saving 12 new artist IDs to /Volumes/Biggy/Discog/artists-rockcorner-db/92-DB.p
Saving 13 new artist IDs to /Volumes/Biggy/Discog/artists-rockcorner-db/68-DB.p
  --> This file is 50.8kB.
Saving 12 new artist IDs name data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/92-Metadata.p
  --> This file is 568B.
Saving 12 new artist IDs media data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/92-MediaMetadata.p
  --> This file is 39.9kB.
Parsing Artist Files For ModVa

  --> This file is 644B.
Saving 15 new artist IDs media data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/97-MediaMetadata.p
  --> This file is 51.4kB.
Parsing Artist Files For ModVal 98
Forcing Reloads of ModVal=98
  Processing 16 files.
	Processed 0/16 files.
Saving 16 new artist IDs to /Volumes/Biggy/Discog/artists-rockcorner-db/98-DB.p
  --> This file is 69.6kB.
Saving 16 new artist IDs name data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/98-Metadata.p
  --> This file is 707B.
Saving 16 new artist IDs media data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/98-MediaMetadata.p
  --> This file is 55.5kB.



ValueError: Cannot find Artist Section

# Albums

In [ ]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
from discogsBase import discogs

disc = discogs()
albs = albums(disc)
#albs.parseAlbumModValFiles(modVal=27)

In [ ]:
def parseAlbums(modVal):
    albs.parseAlbumModValFiles(modVal=modVal)
    #albs.createAlbumModValMetadata(modVal)
    #albs.createAlbumModValMetadata(modVal=modVal)

def parseAlbumsParallel(modVals=None):
    if modVals is None:
        modVals = ["NAN"] + list(range(100))
        #modVals = range(100)
    nProcs = 2
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseAlbums, modVals)
 
    start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
    print("Runing .",end="")
    while not result.ready():
        #print(".",end="")
        time.sleep(10)
    print("")
    
    elapsed(start, cmt)
    return result.get()

In [ ]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
albs = albums(disc)

start,cmt=clock("Parsing All Albums")
start2,cmt2=clock("Parallel Album Parsing")
parseAlbumsParallel()
elapsed(start2, cmt2)
elapsed(start, cmt)

In [ ]:
from fsUtils import removeFile